# Preparing Airtable data for the analysis
These are some interstitial helper scripts to clean some of our manually entered data up and make it easier to work with in the rest of the ETL.

## Main airtable data
This first one needs to be run before 4_filter_id_missing_coord.py

In [1]:
import pandas

LOAD_AIRTABLE = "../data/manual/C. Activity details-Jan 1, 2019 and after.csv"

airtable = pandas.read_csv(LOAD_AIRTABLE)

def make_pt_check(row):
    pt = row["PT - is this a bridge?"]
    if pandas.isna(pt):
        return None
    return pt == "Bridge"

def make_rc_check(row):
    rc1 = row["rosie category"]
    rc2 = row["rc_is_bridge (from activity_id copy)"]
    if pandas.isna(rc1) & pandas.isna(rc2):
        return None
    return ((rc1 in ["Bridge","Pier","swing/draw bridge"])|(rc2 == "bridge"))

def make_af_check(row):
    af1 = row["data_check_bridge"]
    af2 = row["AF_bridge? (from link_to_3)"]
    af3 = row["AF_include? (from activity_id copy)"]
    if pandas.isna(af1) & pandas.isna(af2) & pandas.isna(af3):
        return None
    return True

def ambiguous_facility_cost(nbf):
    if pandas.isna(nbf):
        return None
    if nbf == "possibly no bridge damage":
        return True
    return False
    

airtable["ambiguous_facility_cost"] = airtable["non_bridge_facility--check"].apply(ambiguous_facility_cost)
airtable["pt_check"] = airtable.apply(make_pt_check,axis=1)
airtable["rc_check"] = airtable.apply(make_rc_check,axis=1)
airtable["af_check"] = airtable.apply(make_af_check,axis=1)
airtable["no_false"] = ((airtable["pt_check"]!=False) & (airtable["af_check"]!=False) & (airtable["rc_check"]!=False))
airtable["count_true"] = airtable[["pt_check","af_check","rc_check"]].sum(axis=1)
airtable["passed_review"] = (airtable["no_false"]) & (airtable["count_true"]>=2)
airtable[["activity_id","pt_check","rc_check","af_check","ambiguous_facility_cost","no_false","count_true","passed_review"]].to_csv("../data/manual/manual_bridge_review.csv",index=False)

## Missing coords
This needs to be run before 6_combine_geodata.py

In [2]:
LOAD_CORRECTED_COORDS = "../data/manual/5. allisions_far_from_bridges-Grid view.csv"

# coordinates that were reviewed for being far from bridges
corrected_coords = pandas.read_csv(LOAD_CORRECTED_COORDS,encoding="utf-8")

# rc_coords have been checked against af_coords
corrected_coords = corrected_coords[corrected_coords["rc_coord"].str.strip()!=","] # Taking out uncorrected coords
corrected_coords["lat"] = corrected_coords["rc_coord"].apply(lambda x: float(x.split(",")[0]))
corrected_coords["long"] = corrected_coords["rc_coord"].apply(lambda x: float(x.split(",")[1]))

corrected_coords[["activity_id","lat","long"]].to_csv("../data/manual/corr_coords.csv")

## Corrected coords
This needs to be run before 6_combine_geodata.py

In [3]:
LOAD_CORRECTED_COORDS = "../data/manual/5. allisions_far_from_bridges-Grid view.csv"

# coordinates that were reviewed for being far from bridges
corrected_coords = pandas.read_csv(LOAD_CORRECTED_COORDS,encoding="utf-8")

# rc_coords have been checked against af_coords
corrected_coords = corrected_coords[corrected_coords["rc_coord"].str.strip()!=","] # Taking out uncorrected coords
corrected_coords["lat"] = corrected_coords["rc_coord"].apply(lambda x: float(x.split(",")[0]))
corrected_coords["long"] = corrected_coords["rc_coord"].apply(lambda x: float(x.split(",")[1]))

corrected_coords[["activity_id","lat","long"]].to_csv("../data/manual/corr_coords.csv")